In [ ]:
import pandas as pd
import numpy as np
from statistics import mean, median
from pathlib import Path
import sys
import matplotlib.pyplot as plt


In [ ]:
BASE_PATH = Path.cwd().parent
TESTING_FILES_BASE_PATH = BASE_PATH/"testing"/"files"
sys.path.append(str(BASE_PATH))

In [ ]:
from model.main import EMOTIONS
print(EMOTIONS)

def plot_beliefs(belief_user, belief_character, belief_user_character):
    x = np.arange(len(EMOTIONS))
    width = 0.25  # Slightly smaller width to fit 3 bars

    label1 = "System's belief about User"
    label2 = "System's belief about other character"
    label3 = "System's belief about User's belief about other character"

    fig, ax = plt.subplots()
    bars1 = ax.bar(x - width, belief_user, width, label=label1)
    bars2 = ax.bar(x, belief_character, width, label=label2)
    bars3 = ax.bar(x + width, belief_user_character, width, label=label3)

    ax.set_xlabel('Mental States')
    ax.set_ylabel('Probability')
    ax.set_title('Belief Distribution')
    ax.set_xticks(x)
    ax.set_xticklabels(EMOTIONS)
    ax.legend()

    plt.ylim(0, 1)
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
# Entropy: using log base 2
def entropy(dist):
    dist = np.array(dist)
    dist = dist[dist > 0]  # Avoid log(0)
    return -np.sum(dist * np.log2(dist))

In [ ]:
# Takes data from neg_correlation_emotions, pos_correlation_emotions and uncorrelated_emotions distributions
# Generates stats on distribution divergence (Jensen-Shannon divergence) 
class TestCaseDiffEntropy():
    # Read files and generate stats
    def __init__(self, folder_path_dists, file_path_messages):
        with open(file_path_messages, encoding="utf-8") as f_explicit:
            self.messages = f_explicit.read().split("\n")

        self.num_files = len(self.messages)
        self.System_beliefs_user = [] # dist 1: System's belief of User
        self.System_beliefs_character = [] # dist 2: System's belief of other character
        self.System_beliefs_users_belief_character = [] # dist 3: System's belief of User's belief of other character
        self.diffs_1_2 = []
        self.diffs_1_3 = []
        self.diffs_2_3 = []

        for i in range(self.num_files):
            df = pd.read_csv(folder_path_dists/("message_"+str(i+1)+"_belief_dist.csv"))
            self.System_beliefs_user.append(df["Prob_System_belief_User"])
            self.System_beliefs_character.append(df["Prob_System_belief_Character"])
            self.System_beliefs_users_belief_character.append(df["Prob_System_belief_User_belief_Character"])

            # Jensen-Shannon Divergences
            diff_1_2 = abs(entropy(df["Prob_System_belief_User"]) - entropy(df["Prob_System_belief_Character"]))
            self.diffs_1_2.append(diff_1_2)

            diff_1_3 = abs(entropy(df["Prob_System_belief_User"]) - entropy(df["Prob_System_belief_User_belief_Character"]))
            self.diffs_1_3.append(diff_1_3)

            diff_2_3 = abs(entropy(df["Prob_System_belief_Character"]) - entropy(df["Prob_System_belief_User_belief_Character"]))
            self.diffs_2_3.append(diff_2_3)
            
    def display_stats(self):
        for i in range(self.num_files):
            print("\n------------------------------------------------------------------" \
            "\n"+str(i+1)+"-User message:", self.messages[i])
            plot_beliefs(self.System_beliefs_user[i], self.System_beliefs_character[i], self.System_beliefs_users_belief_character[i])
            print("Entropy Delta - between System's belief of User & System's belief of other character: ", self.diffs_1_2[i])
            
            print("Entropy Delta - between System's belief of User & System's belief of User's belief of other character: ", self.diffs_1_3[i])
            
            print("Entropy Delta - between System's belief of Other character & System's belief of User's belief of other character: ", self.diffs_2_3[i])
            

Get and display all results individually

In [ ]:
DISTRIBUTIONS_BASE_PATH = TESTING_FILES_BASE_PATH/"distributions"
MESSAGES_BASE_PATH = TESTING_FILES_BASE_PATH/"messages"

# Divergence test cases
aligned_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"aligned_emotions", MESSAGES_BASE_PATH/"aligned_emotions.txt")
neither_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"neither_aligned_nor_unaligned_emotions", MESSAGES_BASE_PATH/"neither_aligned_nor_unaligned_emotions.txt")
unaligned_test_case = TestCaseDivergence(DISTRIBUTIONS_BASE_PATH/"unaligned_emotions", MESSAGES_BASE_PATH/"unaligned_emotions.txt")


In [ ]:
# Display individual message stats
print("ENTROPY DELTA STATS \n\n")

print("Prompted to be alinged: \n")
aligned_test_case.display_stats()
print("Prompted to be neither aligned nor unaligned: \n")
neither_test_case.display_stats()
print("Prompted to be unaligned: \n")
unaligned_test_case.display_stats()

High level overview

In [ ]:
def show_aggregated_results_entropy_delta(aligned_test_case, neither_test_case, unaligned_test_case):
    num_files = aligned_test_case.num_files
    labels = [
        'Character & User emotions aligned', 'Character & User emotions not specified', 'Character & User emotions unaligned'
    ]

    # ENTROPY DELTA - between System's belief of User & System's belief of other character
    plt.figure(figsize=(5, 5))
    plt.boxplot([
        aligned_test_case.diffs_1_2,
        neither_test_case.diffs_1_2,
        unaligned_test_case.diffs_1_2,
    ], labels=labels)
    plt.ylabel("Entropy Delta")
    plt.title("ENTROPY DELTA - between System's belief of User & System's belief of other character")
    plt.ylim(0, 0.5)
    plt.xticks(rotation=90)
    plt.show()
    print(f"Means — Aligned: {mean(aligned_test_case.diffs_1_2):.4f}, Neither: {mean(neither_test_case.diffs_1_2):.4f}, Unaligned: {mean(unaligned_test_case.diffs_1_2):.4f}")
    print(f"Medians — Aligned: {median(aligned_test_case.diffs_1_2):.4f}, Neither: {median(neither_test_case.diffs_1_2):.4f}, Unaligned: {median(unaligned_test_case.diffs_1_2):.4f}")
    

    # ENTROPY DELTA - between System's belief of User & System's belief of User's belief of other character
    plt.figure(figsize=(5, 5))
    plt.boxplot([
        aligned_test_case.diffs_1_3,
        neither_test_case.diffs_1_3,
        unaligned_test_case.diffs_1_3,
    ], labels=labels)
    plt.ylabel("Entropy Delta")
    plt.title("ENTROPY DELTA - between System's belief of User &\n System's belief of User's belief of other character")
    plt.ylim(0, 0.5)
    plt.xticks(rotation=90)
    plt.show()
    print(f"Means — Aligned: {mean(aligned_test_case.diffs_1_3):.4f}, Neither: {mean(neither_test_case.diffs_1_3):.4f}, Unaligned: {mean(unaligned_test_case.diffs_1_3):.4f}")
    print(f"Medians — Aligned: {median(aligned_test_case.diffs_1_3):.4f}, Neither: {median(neither_test_case.diffs_1_3):.4f}, Unaligned: {median(unaligned_test_case.diffs_1_3):.4f}")
    

    # ENTROPY DELTA - between System's belief of other character & System's belief of User's belief of other character
    plt.figure(figsize=(5, 5))
    plt.boxplot([
        aligned_test_case.diffs_2_3,
        neither_test_case.diffs_2_3,
        unaligned_test_case.diffs_2_3,
    ], labels=labels)
    plt.ylabel("Entropy Delta")
    plt.title("ENTROPY DELTA - between System's belief of other character &\n System's belief of User's belief of other character")
    plt.ylim(0, 0.5)
    plt.xticks(rotation=90)
    plt.show()
    print(f"Means — Aligned: {mean(aligned_test_case.diffs_2_3):.4f}, Neither: {mean(neither_test_case.diffs_2_3):.4f}, Unaligned: {mean(unaligned_test_case.diffs_2_3):.4f}")
    print(f"Medians — Aligned: {median(aligned_test_case.diffs_2_3):.4f}, Neither: {median(neither_test_case.diffs_2_3):.4f}, Unaligned: {median(unaligned_test_case.diffs_2_3):.4f}")
    


show_aggregated_results_entropy_delta(aligned_test_case, neither_test_case, unaligned_test_case)


In [ ]:
from scipy.stats import kruskal, mannwhitneyu

# Statistical significance

def compute_stats(label, aligned, neither, unaligned):
    print(f"\n--- {label} ---")

    # Kruskal-Wallis Test across all 3 groups
    h_stat, p_value = kruskal(aligned, neither, unaligned)
    print(f"Kruskal-Wallis H-test: H = {h_stat:.4f}, p = {p_value:.4e}")

    # Pairwise Mann-Whitney U Tests
    def pairwise_test(a, b, label_a, label_b):
        u, p = mannwhitneyu(a, b, alternative="two-sided")
        print(f" {label_a} vs {label_b}: Mann–Whitney U = {u}, p = {p:.4e}")

    print("Pairwise Mann-Whitney U tests:")
    pairwise_test(aligned, neither, "Aligned", "Neither")
    pairwise_test(aligned, unaligned, "Aligned", "Unaligned")
    pairwise_test(neither, unaligned, "Neither", "Unaligned")

def show_statistical_results(aligned_test_case, neither_test_case, unaligned_test_case):
    compute_stats("JSD: Belief(User) vs Belief(Other)", aligned_test_case.diffs_1_2, neither_test_case.diffs_1_2, unaligned_test_case.diffs_1_2)
    compute_stats("JSD: Belief(User) vs Belief(User→Other)", aligned_test_case.diffs_1_3, neither_test_case.diffs_1_3, unaligned_test_case.diffs_1_3)
    compute_stats("JSD: Belief(Other) vs Belief(User→Other)", aligned_test_case.diffs_2_3, neither_test_case.diffs_2_3, unaligned_test_case.diffs_2_3)

show_statistical_results(aligned_test_case, neither_test_case, unaligned_test_case)